In [6]:
! pip install gensim 
! pip install pyldavis
! pip install matplotlib




In [7]:
import matplotlib.pyplot as plt 
import logging 
import pandas as pd
import json 
import glob 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import spmatrix 
import pyLDAvis.gensim_models
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
logger = logging.getLogger()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
import glob 
def data_compactor(data_dir: str) -> None:
    """Read files from dir and combine into a dataframe

    Args:
        data_dir (str): name of sub folder to extract data from 

    Returns:
        None: writes the dataframe to a csv file
    """
    data_list = []
    try:
        for name in glob.glob(f"../../data/raw/{data_dir}/*.json"):
            article = json.load(open(name,'r'))
            data_list.append(article)
    except FileNotFoundError as e: 
        logging.error(f"File not found: {e}")
    else:   
        df = pd.DataFrame(data_list)
        req_cols = ['description', 'maintext', 'source_domain', 'title', 'url', 'language', 'date_publish']
        df = df[req_cols]
        df.to_csv(f"../../data/processed/{data_dir}.csv", index=False)
    return

In [9]:
data_compactor("UK")
#data_compactor("MiddleEast")
data_compactor("US")

In [11]:
df_UK = pd.read_csv("../../data/processed/UK.csv")
df_US = pd.read_csv("../../data/processed/US.csv")

In [12]:
df_UK_titles = df_UK["title"]

In [20]:
def get_tfidf_scores(raw_df: pd.DataFrame)-> tuple:
    """Function to generate TF-IDF

    Args:
        raw_df (pd.DataFrame): _description_
    """
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                    stop_words = 'english',
                                    lowercase = True,
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    max_df = 0.5, 
                                    min_df = 0.01
                                    )
    tf = tf_vectorizer.fit_transform(raw_df)
    tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
    tfidf= tfidf_vectorizer.fit_transform(raw_df)
    return tf,tfidf,tfidf_vectorizer

In [27]:
def lda_topic_modelling(tf: spmatrix, tfidf: spmatrix) -> tuple:
    """Function to run LDA on tf and tfidf matrices for the dataframes

    Args:
        tf (spmatrix): Term Frequency Matrix 
        tfidf (spmatrix): TFIDF Matrix 

    Returns:
        : _description_
    """
    # initialse model for term frequency matrix
    lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
    lda_tf.fit(tf)

    # initialise model for tfidf matrix 
    lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
    lda_tfidf.fit(tfidf)

    return lda_tf, lda_tfidf

In [28]:
tf, tf_idf, tf_vectorizer = get_tfidf_scores(df_UK_titles)
lda_tf, lda_tfidf = lda_topic_modelling(tf, tf_idf)



In [29]:
for topic_idx, topic in enumerate(lda_tfidf.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([tf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-10 - 1:-1]]))

Topic 0:
happened aid gaza israel war says biden say hostages president
Topic 1:
ukraine russia attack putin soldiers war american nato ukrainian troops
Topic 2:
israel hamas idf new war gaza missile leader terror terrorists
Topic 3:
market research billion reach forecast stratview says japan aircraft korea
Topic 4:
gaza israeli hospital israel forces palestinians killed shifa hostage says
Topic 5:
russian army west people hezbollah dead bank border israeli plane
Topic 6:
gaza ceasefire calls netanyahu palestinian israel minister hamas shows civilians
Topic 7:
day pro conflict palestine london world group video know state
Topic 8:
military china support east talks kyiv navy ukraine weapons middle
Topic 9:
yemen sea red british houthi iran attacks drone ship houthis


In [30]:
pyLDAvis.lda_model.prepare(lda_tf, tf, tf_vectorizer)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.254049 -0.018779       1        1  17.589324
4     -0.152808 -0.036265       2        1  13.833014
5      0.016283 -0.170967       3        1  10.806241
1      0.024094 -0.024299       4        1  10.724191
6     -0.229720 -0.085981       5        1   9.583251
8      0.282303 -0.184680       6        1   8.754076
0      0.066618  0.101587       7        1   7.799667
3      0.092273  0.398593       8        1   7.730044
7     -0.116756  0.061146       9        1   7.238628
9      0.271762 -0.040354      10        1   5.941565, topic_info=        Term        Freq       Total Category  logprob  loglift
44      gaza  364.000000  364.000000  Default  30.0000  30.0000
47     hamas  229.000000  229.000000  Default  29.0000  29.0000
137  ukraine  121.000000  121.000000  Default  28.0000  28.0000
62    israel  316.000000  316.000000  Default  27.0000  27.0000
63   israeli  162.000000  162.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
109  refugee    0.109134   14.076703  Topic10  -8.0657  -2.0365
71   largest    0.109138   14.086816  Topic10  -8.0656  -2.0372
62    israel    0.508157  316.848307  Topic10  -6.5275  -3.6122
58       idf    0.140141   64.022527  Topic10  -7.8156  -3.3011
121     shot    0.109466   16.082887  Topic10  -8.0626  -2.1667

[374 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.138630       aid
0         5  0.083178       aid
0         7  0.748600       aid
1         2  0.144834       air
1         3  0.181043       air
...     ...       ...       ...
148       4  0.213309      year
148       9  0.170647      year
149       6  0.758787     yemen
149      10  0.222084     yemen
150       7  0.973089  zelensky

[363 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 6, 2, 7, 9, 1, 4, 8, 10])